Importing libraries

In [ ]:
import numpy as np
import scipy as sp
import scipy.ndimage as ndimage
import math
from skimage import exposure, filters
from matplotlib import pyplot as plt
import cv2
import matplotlib.pyplot as plt
from skimage.transform import hough_circle, hough_circle_peaks, hough_ellipse
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.segmentation import active_contour
from skimage.filters import gaussian
import argparse
from google.colab.patches import cv2_imshow

Preprocessing

In [ ]:
def resize(img):
    width = 1024
    height = 720
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)


def rgb2Blue(img):
    b, g, r = cv2.split(img)
    return b


def rgb2Red(img):
    b, g, r = cv2.split(img)
    return r


def rgb2Green(img):
    b, g, r = cv2.split(img)
    return g


def rgb2Gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def rgb2lab(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2LAB)


#############preprocess###########
#Image is split on B,G,R channel
#Red channel is isolated
#Smoothing over the red channel is applied
#Sharpening and Equalization to te image are applied
#A morph closing is applied to remove artifacts
def preprocess(img):
    b, g, r = cv2.split(img)
    gray = rgb2Red(img)
    gray_blur = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.addWeighted(gray, 1.5, gray_blur, -0.5, 0, gray)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
    gray = ndimage.grey_closing(gray, structure=kernel)
    gray = cv2.equalizeHist(gray)

    return gray


#############getROI##############
#Image is resized
#We take green channel and smooth it
#Opening is done to remove artifacts, in order to preserve only BRIGHTEST elements
#Now we get the most bright pixel position
#We return that position in a 110x110 window
#It is actually a simple way to detect the optic disc, but it works so..

def getROI(image):
    image_resized = resize(image)
    b, g, r = cv2.split(image_resized)
    g = cv2.GaussianBlur(g, (15, 15), 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    g = ndimage.grey_opening(g, structure=kernel)
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(g)

    x0 = int(maxLoc[0]) - 110
    y0 = int(maxLoc[1]) - 110
    x1 = int(maxLoc[0]) + 110
    y1 = int(maxLoc[1]) + 110

    return image_resized[y0:y1, x0:x1]


def getValue(img):
    shapeRow = img.shape[0]
    shapeCol = img.shape[1]
    x = 0
    y = 0
    acu = 0
    maxloc = []
    for i in range(shapeRow):
        for j in range(shapeCol):
            (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(img[i - 15:j - 15, i + 15:j + 15])
            value = maxVal
            if value > acu:
                acu = value
                maxloc = maxLoc
    return maxloc

def checkSide(img):
    shapeRow = img.shape[0]
    shapeCol = img.shape[1]
    if cv2.countNonZero(img[:, 0:int(shapeCol / 2)]) > cv2.countNonZero(img[:, int(shapeCol / 2):shapeCol]):
        return True
    else:
        return False


def checkHigh(img):
    shapeRow = img.shape[0]
    shapeCol = img.shape[1]
    if cv2.countNonZero(img[0:int(shapeRow / 2), :]) > cv2.countNonZero(img[int(shapeRow / 2):shapeRow, :]):
        return True
    else:
        return False

def canny(img, sigma):
    v = np.mean(img)
    sigma = sigma
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(img, lower, upper)
    return edged


def hough(edged, limm, limM):
    hough_radii = np.arange(limm, limM, 1)
    hough_res = hough_circle(edged, hough_radii)
    return hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1)

Hough transform

In [ ]:
#Here we start the process

image = cv2.imread('Image.jpg')
print('input image')
cv2_imshow(image)
roi = getROI(image)
preprocessed_roi = preprocess(roi)

# im2, contours, hierarchy = cv2.findContours(segmented, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
##We get our ROI and then we preprocess it.
##After preprocess, we apply a canny border algorithm
##We dilate the borders in order to make it easy to detect with hough

edged = canny(preprocessed_roi, 0.22)
kernel = np.ones((3, 3), np.uint8)
edged = cv2.dilate(edged, kernel, iterations=3)
accums, cx, cy, radii = hough(edged, 55, 80)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    try:
        roi[circy, circx] = (220, 20, 20)
    except:
        continue

init = np.array([circx, circy]).T

print('preprocessed image of range of interest')
cv2_imshow(preprocessed_roi)
print('output image with detection of optic disc(denoted in blue circle)')
cv2_imshow(roi)

In [ ]:
print(roi)
print(roi.shape)

Growcut algorithm

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def growcut(image, seed_mask, num_iter=5):
    """
    Implementation of the GrowCut algorithm for segmentation.
    :param image: grayscale input image.
    :param seed_mask: binary mask specifying the initial seed regions.
    :param num_iter: number of iterations to run the algorithm.
    :return: binary mask with the segmented regions.
    """
    # Set up the markers for the GrowCut algorithm
    markers = np.zeros_like(seed_mask)
    markers[seed_mask == 255] = 1
    markers[seed_mask == 0] = 2

    # Run the GrowCut algorithm
    for i in range(num_iter):
        fg = np.ma.masked_array(image, markers != 1)
        bg = np.ma.masked_array(image, markers != 2)

        fg_mean = np.mean(fg)
        bg_mean = np.mean(bg)

        markers[image > fg_mean] = 1
        markers[image < bg_mean] = 2

    # Return the segmented binary mask
    segmented_mask = np.zeros_like(seed_mask)
    segmented_mask[markers == 1] = 255
    return segmented_mask

# Load the input image and convert it to grayscale
input_image = cv2.imread('Image_01L.jpg')
gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

# Apply a median filter to reduce noise
gray_image = cv2.medianBlur(gray_image, 5)

# Compute the Sobel gradients along x and y
sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0)
sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1)

# Compute the magnitude and direction of the gradients
mag = np.sqrt(sobel_x ** 2 + sobel_y ** 2)
angle = np.arctan2(sobel_y, sobel_x)

# Threshold the magnitude to obtain an initial seed mask
seed_mask = np.zeros_like(gray_image)
seed_mask[mag > np.mean(mag)] = 255

# Run the GrowCut algorithm to segment the optic disc
segmented_mask = growcut(mag, seed_mask)

# Save the segmented mask to a file
cv2.imwrite('segmented_mask.png', segmented_mask)
cv2_imshow(segmented_mask)

Region Growing algorithm

In [ ]:
import cv2
import numpy as np
from skimage.filters import frangi
from scipy import ndimage as ndi
from google.colab.patches import cv2_imshow

# Load the retinal image and pre-process it
img = cv2.imread('retinal_image.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
img = cv2.equalizeHist(img)
img = cv2.medianBlur(img, 5)

# Segment the optic disc region using a thresholding-based approach
od_mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 101, 40)

# Perform vessel segmentation using Frangi filter
vesselness = frangi(img)

# Generate a seed point inside the optic disc region
od_center = ndi.measurements.center_of_mass(od_mask)
seed_point = (int(od_center[0]), int(od_center[1]))

# Define the region growing criteria

def region_growing(img, seed_point, threshold):
    rows, cols = img.shape
    visited = np.zeros_like(img)
    region = np.zeros_like(img)
    region_size = 1
    intensity_diff = 0
    neighbors_list = [seed_point]

    # Calculate the mean intensity of the seed point's neighbors
    mean_intensity = float(img[seed_point])

    # Loop through neighbors until the region stops growing
    while(len(neighbors_list) > 0):
        # Pop the last pixel from the neighbors list
        pixel = neighbors_list.pop()

        # Check if pixel has already been visited
        if visited[pixel] == 1:
            continue
        # Add pixel to region and mark as visited
        region[pixel] = 255
        visited[pixel] = 1
        region_size += 1

        # Calculate the intensity difference between the pixel and its neighbors
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i == 0 and j == 0:
                    continue
                if pixel[0]+i < 0 or pixel[0]+i >= rows or pixel[1]+j < 0 or pixel[1]+j >= cols:
                    continue
                if visited[pixel[0]+i, pixel[1]+j] == 1:
                    continue

                neighbor_intensity = float(img[pixel[0]+i, pixel[1]+j])
                intensity_diff = abs(neighbor_intensity - mean_intensity)

                # If intensity difference is below the threshold, add neighbor to the region and the neighbors list
                if intensity_diff < threshold:
                    neighbors_list.append((pixel[0]+i, pixel[1]+j))
                    # Update the mean intensity of the region
                    mean_intensity = (mean_intensity*region_size + neighbor_intensity)/(region_size+1)

    return region

# Run the region growing algorithm from the seed point
od_region = region_growing(vesselness, seed_point, 0.1)

# Post-process the region growing result
od_region = np.uint8(od_region)
od_region = cv2.medianBlur(od_region, 5)
od_region = cv2.morphologyEx(od_region, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
od_region = cv2.morphologyEx(od_region, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))

# Display the results
cv2_imshow(img)
cv2_imshow(od_mask)
cv2_imshow(vesselness)
cv2_imshow(od_region)